In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split

In [ ]:
def XyData(csv_path, X_path, y_path):
    usecols = ['content', 'type']
    df = pd.read_csv(csv_path, usecols=usecols)
    
    df.dropna(subset=['type'], inplace = True) #remove empty labels
    df = df.loc[df.content.apply(type) == str]
    #df.drop(df[df['type'] == 'unknown'].index, inplace=True) #remove 'unknown' label
    df.reset_index(drop=True, inplace=True) #reset index

    l = ['fake']
    y = pd.DataFrame({'result' : np.where(df['type'].isin(l), 1, 0)}) #[1, 0] = [fake, not fake]

    unique_words = Counter()
    df['content'].str.split().apply(unique_words.update)
    unique_words.subtract(unique_words) #keep keys, reset count
 
    frames = []  
    for row in range(len(df)):
        words = unique_words.copy()
        words.update(df.loc[row,"content"].split())
        frames += [pd.DataFrame(words, index=[row])]
    X = pd.concat(frames)
         
    X.to_csv(X_path, index=False)
    y.to_csv(y_path, index=False)

XyData('..\\data\\clean_995,000_rows.csv', 'X_995k.csv', 'y_995k.csv')
#XyData('..\\cleaned_news_sample.csv', 'X.csv', 'y.csv')

In [ ]:
def Split(X_path, y_path):
    df_X = pd.read_csv(X_path)
    df_y = pd.read_csv(y_path)
    X_train, X_test, y_train, y_test = train_test_split(df_X,
                                                        df_y,
                                                        train_size = 0.8,
                                                        random_state = 0,
                                                        shuffle = False,
                                                        stratify = None)

    X_test, X_val, y_test, y_val = train_test_split(X_test,
                                                    y_test,
                                                    test_size= 0.5,
                                                    random_state = 0,
                                                    shuffle = False,
                                                    stratify = None)

    return X_train, X_test, X_val, y_train, y_test, y_val
X_train, X_test, X_val, y_train, y_test, y_val = Split('X.csv', 'y.csv')

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

model = LinearRegression()
fit = model.fit(X_train, y_train['label'])
y_pred = fit.predict(X_test)
y_pred = [round(x,0) for x in y_pred]

mse = mean_squared_error(y_test['label'], y_pred)
print("LinearRegression MSE: ", mse)
acc = accuracy_score(y_test['label'], y_pred)
print("LinearRegression accuracy: ", acc)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

model = LogisticRegression(random_state=0, max_iter=1000)
fit = model.fit(X_train, y_train['label'])
y_pred = fit.predict(X_test)
y_pred = [round(x,0) for x in y_pred]

mse = mean_squared_error(y_test['label'], y_pred)
print("LinearRegression MSE: ", mse)
acc = accuracy_score(y_test['label'], y_pred)
print("LinearRegression accuracy: ", acc)

In [ ]:
from sklearn.svm import SVC 
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X_train, y_train['label'])
y_pred = clf.predict(X_test)

mse = mean_squared_error(y_test['label'], y_pred)
print("LinearRegression MSE: ", mse)
acc = accuracy_score(y_test['label'], y_pred)
print("LinearRegression accuracy: ", acc)

In [ ]:
y = pd.read_csv('y_955K.csv')
c = Counter()
c.update(y_test['label'])
print(c[1]/(c[1]+c[0]))